In [1]:
import sys

sys.path.append('../')

In [2]:
import pickle
from glob import glob
import os
import yaml
from easydict import EasyDict as edict
import numpy as np
import torch

In [3]:
from model.resnet_xor import *
from model.resnet import *

In [4]:
from torchvision import transforms, datasets
from torch.utils.data import DataLoader, random_split, ConcatDataset

In [5]:
transform = transforms.Compose(
    [transforms.ToTensor(), transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])])
test_dataset = datasets.CIFAR10(root='../xor_neuron_data/data',
                                train=False,
                                transform=transform,
                                download=True)

Files already downloaded and verified


In [6]:
test_loader = DataLoader(dataset=test_dataset,
                                 batch_size=16,
                                 shuffle=False)

In [7]:
imgs, target = next(iter(test_loader))

In [8]:
# config_file = glob('../config/resnet/0207_resnet_quad_v2_rtx30/4.yaml')[0]
# config = edict(yaml.load(open(config_file, 'r'), Loader=yaml.FullLoader))

config_file = glob('../config/resnet/xor_resnet.yaml')[0]
config = edict(yaml.load(open(config_file, 'r'), Loader=yaml.FullLoader))

control_config_file = glob('../config/control_model/resnet20.yaml')[0]
control_config = edict(yaml.load(open(control_config_file, 'r'), Loader=yaml.FullLoader))

In [9]:
config.model.num_blocks = [3,3,3]

In [10]:
model = ResNet20_Xor(config)

In [11]:
model

ResNet_Xor(
  (loss_func): CrossEntropyLoss()
  (inner_net): Sequential(
    (fc1): Linear(in_features=2, out_features=16, bias=True)
    (relu1): ReLU()
    (fc2): Linear(in_features=16, out_features=16, bias=True)
    (relu2): ReLU()
    (fc3): Linear(in_features=16, out_features=1, bias=True)
  )
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layer1): Sequential(
    (0): BasicBlock_InnerNet(
      (inner_net): Sequential(
        (fc1): Linear(in_features=2, out_features=16, bias=True)
        (relu1): ReLU()
        (fc2): Linear(in_features=16, out_features=16, bias=True)
        (relu2): ReLU()
        (fc3): Linear(in_features=16, out_features=1, bias=True)
      )
      (conv1): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)

In [12]:
out, loss, _ = model(imgs, target)

In [13]:
out

tensor([[-0.0926, -0.0960,  0.0124, -0.0494, -0.0905, -0.1491,  0.0513,  0.0211,
          0.0561,  0.0709],
        [-0.0908, -0.0955,  0.0129, -0.0494, -0.0905, -0.1488,  0.0518,  0.0212,
          0.0581,  0.0704],
        [-0.0914, -0.0962,  0.0115, -0.0491, -0.0907, -0.1475,  0.0518,  0.0220,
          0.0585,  0.0710],
        [-0.0921, -0.0964,  0.0113, -0.0483, -0.0886, -0.1485,  0.0524,  0.0211,
          0.0579,  0.0715],
        [-0.0923, -0.0953,  0.0124, -0.0489, -0.0908, -0.1507,  0.0512,  0.0217,
          0.0572,  0.0709],
        [-0.0923, -0.0956,  0.0128, -0.0494, -0.0904, -0.1484,  0.0522,  0.0200,
          0.0573,  0.0712],
        [-0.0924, -0.0945,  0.0121, -0.0487, -0.0907, -0.1483,  0.0518,  0.0209,
          0.0575,  0.0713],
        [-0.0923, -0.0960,  0.0115, -0.0496, -0.0910, -0.1494,  0.0517,  0.0213,
          0.0580,  0.0713],
        [-0.0932, -0.0958,  0.0121, -0.0485, -0.0895, -0.1486,  0.0517,  0.0223,
          0.0568,  0.0713],
        [-0.0927, -

In [14]:
loss

tensor(2.2873, grad_fn=<NllLossBackward>)

In [15]:
origin_resnet = resnet20(control_config)

In [16]:
origin_resnet

ResNet(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (activation_fnc): ReLU()
      (shortcut): Sequential()
    )
    (1): BasicBlock(
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(16, e